In [1]:
map_html = '''
<!DOCTYPE html>
<html>
<head>
    <link rel="stylesheet" href="https://unpkg.com/leaflet@1.5.1/dist/leaflet.css" integrity="sha512-xwE/Az9zrjBIphAcBb3F6JVqxf46+CDLwfLMHloNu6KEQCAWi6HcDUbeOfBIptF7tcCzusKFjFw2yuvEpDL9wQ==" crossorigin=""/>
    <script src="https://unpkg.com/leaflet@1.5.1/dist/leaflet.js" integrity="sha512-GffPMF3RvMeYyc1LWMHtK8EbPv0iNZ8/oTtHPx9/cc2ILxQ+u905qIwdpULaqDkyBKgOaB57QTMg7ztg8Jm2Og==" crossorigin=""></script>
    <script src="http://guoxiaokang.net/js/leaflet.rotatedMarker.js"> </script>
    <script src="http://guoxiaokang.net/js/axios.min.js"> </script>
    <style>
        #restartButton {
            font-size: 1.5em;
            position:relative;
            left: 50%;
            transform: translate(-50%, 10%)
        }
    </style>
</head>
<body>
<div id="mapid" style="width: 100%; height: 600px;"></div>
<script>
    var body = document.body,
        html = document.documentElement;
    var height = Math.max( body.scrollHeight,
                           body.offsetHeight,
                           html.clientHeight,
                           html.scrollHeight,
                           html.offsetHeight );
    document.getElementById("mapid").style.height = height*0.7+"px";
    </script>
<div style="font-size: 1.5em; text-align: center; display:none">The GPS tracker would stop working unless it received new GPS position from Server.</div>
<button onclick="trackingSwitch()" id="restartButton"> NA </button>
<script>
    var map = L.map('mapid').setView([40.7611000, -73.9506000], 16);
    L.tileLayer('https://api.tiles.mapbox.com/v4/{id}/{z}/{x}/{y}.png?access_token=pk.eyJ1IjoibWFwYm94IiwiYSI6ImNpejY4NXVycTA2emYycXBndHRqcmZ3N3gifQ.rJcFIG214AriISLbB6B5aw', {
        maxZoom: 18,
        attribution: 'Map data &copy; <a href="https://www.openstreetmap.org/">OpenStreetMap</a> contributors, <a href="https://creativecommons.org/licenses/by-sa/2.0/">CC-BY-SA</a>, Imagery © <a href="https://www.mapbox.com/">Mapbox</a>',
        id: 'mapbox.streets'
    }).addTo(map);
    var roverIcon = L.icon({
        iconUrl: 'http://guoxiaokang.net/icon/rover.png', // https://www.iconfinder.com/search/?q=find
        iconSize: [32, 32],
        iconAnchor: [16, 16],
        popupAnchor: [0, 0],
        shadowUrl: '',
        shadowSize: [0, 0],
        shadowAnchor: [0, 0]
    });
    var roverLayer;
    var timestamp = "newcomer";
    var updateTimer;  // update the map every few seconds
    var stopTimer; // if we cannot get new GPS position in a while, stop update the map.
    function updateGPS() {
        axios.get('http://guoxiaokang.net:9999/gps', {params: {timestamp: timestamp}})//
            .then(response => {
                // console.log(response.data);
                var rover = response.data.rover;
                if (typeof rover != "undefined") {
                    if (typeof roverLayer != "undefined") {
                        map.removeLayer(roverLayer);
                    }
                    roverLayer = L.geoJSON(rover, {
                        pointToLayer: function (geoJsonPoint, latlng) {
                            return L.marker(latlng, { icon: roverIcon })
                        }
                    });
                    roverLayer.addTo(map);
                    map.setView(rover.properties.newestViewCenter, 16);
                    if (timestamp != rover.properties.newestTimestampServerSide) {
                        timestamp = rover.properties.newestTimestampServerSide;
                        startTracking();
                    }
                }
                var trail = response.data.trail;
                if (typeof trail != "undefined") {
                    L.geoJSON(trail).addTo(map);
                }
            })
    };
    function startTracking() {
        document.getElementById("restartButton").innerHTML="Tracking ...";
        clearInterval(updateTimer);
        clearTimeout(stopTimer);
        updateTimer = setInterval(updateGPS, 1000)
        stopTimer = setTimeout(function () {
            clearInterval(updateTimer);
            document.getElementById("restartButton").innerHTML="No new GPS Position, tracking stopped automatically but you restart it by clicking me ~";
            //alert("No new GPS Position, tracker is stopped in 20 seconds unless you restart it ~");
        }, 10000);
    };
    var tracking = true;
    function trackingSwitch() {
        if (tracking) {
            clearInterval(updateTimer);
            clearTimeout(stopTimer);
            document.getElementById("restartButton").innerHTML="Tracking is stopped manually and you can restart it by clicking me ~";
            tracking = false;
        } else {
            startTracking();
            tracking = true;
        }
    }
    startTracking();
    //L.marker([40.7611980, -73.9639120], {icon: scooter, rotationAngle: 30}).addTo(map).bindPopup("<b>Scooter</b>")
</script>
</body>
</html>
'''

In [2]:
import pynmea2, datetime 
def parseGPRMC(GPRMC):
    GPRMC = pynmea2.parse(GPRMC)
    lon, lat = GPRMC.lon, GPRMC.lat
    lat = float(lat[:2]) + float(lat[2:])/60
    lon = float(lon[:3]) + float(lon[3:])/60
    if GPRMC.lat_dir == 'S': lat *= -1 
    if GPRMC.lon_dir == 'W': lon *= -1  
    isoformat = datetime.datetime.combine(GPRMC.datestamp, GPRMC.timestamp).isoformat()
    return f'{lat:.7f}',f'{lon:.7f}', isoformat

In [ ]:
def updateRoverPosition(response, coords, newest_timestamp_server_side): 
    response.update({
        "rover": {
            "type": "Feature", 
            "geometry": {
                "type": "Point",
                "coordinates": coords[-1]
            }, 
            "properties": {
                "newestViewCenter": coords[-1][::-1], 
                "newestTimestampServerSide": newest_timestamp_server_side, 
            },
        }
    })
def updateRoverTrail(response, coords): 
    response.update({
        "trail": {
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": coords
            }
        }
    })

In [ ]:
import pandas as pd
df = pd.DataFrame(columns=['longitude', 'latitude' , 'timestamp'])  
from flask import Flask, request, render_template_string, jsonify 
from flask_cors import CORS 
app = Flask(__name__) 
CORS(app)   
import logging
log = logging.getLogger('werkzeug')
log.setLevel(logging.ERROR) 

@app.route('/gps', methods=['GET', 'POST'])
def gps():
    global df
    if request.method == 'GET':  
        response = {}
        timestamp_server_side = df.to_dict('list')['timestamp'] 
        if timestamp_server_side: # return empty result if no GPS record yet
            newest_timestamp_client_side = request.args.get('timestamp')
            newest_timestamp_server_side = timestamp_server_side[-1].isoformat() 
            if newest_timestamp_client_side == 'newcomer':
                coords = df[['latitude', 'longitude']].values.tolist()
                updateRoverPosition(response, coords, newest_timestamp_server_side)
                if len(coords) > 1: 
                    updateRoverTrail(response, coords) 
            elif newest_timestamp_client_side == newest_timestamp_server_side:# if no new GPS record yet, return empty result
                pass
            elif newest_timestamp_client_side != newest_timestamp_server_side:  
                newest_timestamp_client_side = datetime.datetime.strptime(newest_timestamp_client_side, "%Y-%m-%dT%H:%M:%S")
                coords = df[df['timestamp']>= newest_timestamp_client_side][['latitude', 'longitude']].values.tolist()  
                updateRoverPosition(response, coords, newest_timestamp_server_side)
                updateRoverTrail(response, coords)   
        return jsonify(response)
    elif request.method == 'POST':
        GPRMC = request.form.get('GPRMC') 
        longitude, latitude, timestamp = parseGPRMC(GPRMC)  
        df = df.append({'longitude':longitude, 'latitude':latitude, 'timestamp':datetime.datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S")}, ignore_index=True)
        return 'GPRMC Msg Received!'
    

@app.route('/map', methods=['GET'])
def MAP():
    return render_template_string(map_html)

app.run('0.0.0.0', 9999)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
